# Congestion Finder | Integration Example

This Notebook goes through all the steps of the _congestion finder_ module, with exception of the database connections. Each step builds on the output of the previous step. Therefore, this **Congestion Finder | Integration Example** documents the integration of all classes and functions in the module. Together with the **Congestion Finder | Unit Example**, it serves as documentation for the module, both to allow users and developers to get familiar with the code.

## Initialization

Tho initialize the module, several properties need to be set and modules need to be imported.

### Properties

Usually, the properties are set using a _properties file_. Here, we will define the directly.

In [ ]:
date = "20171120"
roadNumber = 2
roadsFileName = "../tests/data/BPS_20171120.txt"
detectionsFileName = "../tests/data/A2_20171120.txt"
outputDirectory = "../tests/data"
speedThreshold = 65
flowThreshold = 40
spaceSmoothing = 10
timeSmoothing = 20
marginSpace = spaceSmoothing  # probably always the same
marginTime = timeSmoothing  # probably always the same

### Imports

The following list of modules need to be imported. 

In [ ]:
import logging

import numpy
import matplotlib.pyplot

import sys
sys.path.append("../")
import congestionfinder
import congestionfinder.bpsdetector
import congestionfinder.road
import congestionfinder.detection
import congestionfinder.speedflow
import congestionfinder.congestion
import patchfinder.patch

### Logging

During this example, we will log at _debug_ level, to get the most information about the code. We will include the time at the millisecond level, so that we can see the process time of each step.

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
consoleHandler = logging.StreamHandler()
consoleHandler.setFormatter(formatter)
logger.addHandler(consoleHandler)

## Roads

The first step to find congestion is to load the road network. This is done by obtaining the list of all active BPS detectors. In this example, we first read a list of BPS codes from a CSV file. We print the list of codes, which are strings of 20 hexadecimal characters.

In [ ]:
bpsCodes = congestionfinder.bpsdetector.readCSVToBPSCodes(roadsFileName)

In [ ]:
for bpsCode in bpsCodes:
    print(bpsCode)

The BPS codes can be parsed to a new class called _Road_. To find out more about this class, please look at the **Congestion Finder | Unit Example** Notebook. We print the list of all roads, sorted by its number of detectors.

In [ ]:
roads = congestionfinder.road.parseBPSCodesToRoads(bpsCodes)

In [ ]:
for value in sorted(roads.values(), key = lambda x:len(x.getBPSDetectors())):
    print(value)

As we will concentrate on a single road in the example, we define the variable _road_ here for future use.

In [ ]:
road = roads[roadNumber]

## Speed and Flows

In [ ]:
detections = congestionfinder.detection.readCSVToDetections(detectionsFileName)

In [ ]:
iterator = iter(detections)
for i in range(min(len(detections), 300)):
    detection = next(iterator)
    print(detection)

In [ ]:
speeds, flows, minSpaceIndex, maxSpaceIndex, minTimeIndex, maxTimeIndex = congestionfinder.speedflow.parseDetectionsToSpeedsAndFlows(detections, road)

In [ ]:
print(speeds.shape)
print(flows.shape)
print(minSpaceIndex)
print(maxSpaceIndex)
print(minTimeIndex)
print(maxTimeIndex)
matplotlib.pyplot.imshow(speeds, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()
matplotlib.pyplot.imshow(flows, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

In [ ]:
speedsWorkingDetectors, flowsWorkingDetectors, maskWorkingDetectors = congestionfinder.speedflow.removeMissingDetectors(speeds, flows) ## Do something with Mask

In [ ]:
print(speedsWorkingDetectors.shape)
print(flowsWorkingDetectors.shape)
print(sum(maskWorkingDetectors) / (sum(maskWorkingDetectors) + sum(~maskWorkingDetectors)))
matplotlib.pyplot.imshow(speedsWorkingDetectors, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()
matplotlib.pyplot.imshow(flowsWorkingDetectors, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

In [ ]:
speedsHighFlow, flowsHighFlow, maskHighFlow = congestionfinder.speedflow.removeLowFlowTimes(speedsWorkingDetectors, flowsWorkingDetectors) ## Do something with Mask

In [ ]:
print(speedsHighFlow.shape)
print(flowsHighFlow.shape)
print(sum(maskHighFlow) / (sum(maskHighFlow) + sum(~maskHighFlow)))
matplotlib.pyplot.imshow(speedsHighFlow, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()
matplotlib.pyplot.imshow(flowsHighFlow, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

## Congestion

In [ ]:
congestions = congestionfinder.congestion.parseSpeedFlowsToCongestions(speedsHighFlow, flowsHighFlow, speedThreshold, flowThreshold)

In [ ]:
print(congestions.shape)
matplotlib.pyplot.imshow(congestions, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

In [ ]:
congestionsWithoutMissingValues = congestionfinder.congestion.interpolateMissingValues(congestions)

In [ ]:
print(congestionsWithoutMissingValues.shape)
matplotlib.pyplot.imshow(congestionsWithoutMissingValues, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

In [ ]:
congestionsSmoothed = congestionfinder.congestion.applySmoothingFilter(congestionsWithoutMissingValues, spaceSmoothing, timeSmoothing)

In [ ]:
print(congestionsSmoothed.shape)
matplotlib.pyplot.imshow(congestionsSmoothed, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

## Congestion Patch Finder

In [ ]:
congestionsBoolean = congestionsSmoothed < 1

In [ ]:
congestionPatches = patchfinder.patch.findPatches(congestionsBoolean)

In [ ]:
for congestionPatch in congestionPatches:
    print(congestionPatch)

In [ ]:
congestionfinder.congestion.plotCongestionsWithPatches(congestionsSmoothed, congestionPatches)
congestionfinder.congestion.plotCongestionsWithPatches(congestionsBoolean, congestionPatches)

In [ ]:
congestionPatchesFiltered = patchfinder.patch.filterLargePatches(congestionPatches)

In [ ]:
for congestionPatch in congestionPatchesFiltered:
    print(congestionPatch)

In [ ]:
congestionfinder.congestion.plotCongestionsWithPatches(congestionsSmoothed, congestionPatchesFiltered)
congestionfinder.congestion.plotCongestionsWithPatches(congestionsBoolean, congestionPatchesFiltered)

In [ ]:
speedFlowPatches = congestionfinder.speedflow.unmaskPatches(congestionPatchesFiltered, maskWorkingDetectors, maskHighFlow)

In [ ]:
for speedFlowPatch in speedFlowPatches:
    print(speedFlowPatch)

In [ ]:
congestionfinder.congestion.plotCongestionsWithPatches(speeds, speedFlowPatches)

In [ ]:
speedFlowPatchesWithMargins = congestionfinder.speedflow.addMargins(speedFlowPatches, marginSpace, marginTime, minSpaceIndex, maxSpaceIndex, minTimeIndex, maxTimeIndex)

In [ ]:
for speedFlowPatch in speedFlowPatchesWithMargins:
    print(speedFlowPatch)

In [ ]:
congestionfinder.congestion.plotCongestionsWithPatches(speeds, speedFlowPatchesWithMargins)

In [ ]:
congestionfinder.speedflow.writeSpeedsAndFlowsToCSV(speeds, flows, speedFlowPatchesWithMargins, outputDirectory, date, road)

End of Notebook